In [1]:
import os
import cv2
import numpy as np

In [2]:
features=[]
target=[]
for x in range(0,43):
    totfolder=os.listdir("E:/Datasets/traffic sign" + "/" + str(x))
    for y in totfolder:
        image=cv2.imread("E:/Datasets/traffic sign" + "/" + str(x) + "/" + y)
        features.append(image)
        target.append(x)
    print("FOLDER" , x)   

FOLDER 0
FOLDER 1
FOLDER 2
FOLDER 3
FOLDER 4
FOLDER 5
FOLDER 6
FOLDER 7
FOLDER 8
FOLDER 9
FOLDER 10
FOLDER 11
FOLDER 12
FOLDER 13
FOLDER 14
FOLDER 15
FOLDER 16
FOLDER 17
FOLDER 18
FOLDER 19
FOLDER 20
FOLDER 21
FOLDER 22
FOLDER 23
FOLDER 24
FOLDER 25
FOLDER 26
FOLDER 27
FOLDER 28
FOLDER 29
FOLDER 30
FOLDER 31
FOLDER 32
FOLDER 33
FOLDER 34
FOLDER 35
FOLDER 36
FOLDER 37
FOLDER 38
FOLDER 39
FOLDER 40
FOLDER 41
FOLDER 42


In [3]:
features=np.array(features)
target=np.array(target)

In [4]:
features.shape


(34799, 32, 32, 3)

In [5]:
target.shape

(34799,)

In [6]:
from sklearn.model_selection import train_test_split

fea_train, fea_test, tar_train, tar_test = train_test_split(features,target,test_size=0.2)

In [7]:
def processing(image):
    
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = image/255

In [8]:
fea_test.shape

(6960, 32, 32, 3)

In [9]:
tar_test.shape

(6960,)

In [10]:
tar_train.shape

(27839,)

In [11]:
fea_train.shape

(27839, 32, 32, 3)

In [12]:
def preprocessing(image):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image=image/255

    
    
    return image

In [13]:
fea_train=np.array(list(map(preprocessing,fea_train)))

In [14]:
fea_train.shape

(27839, 32, 32)

In [15]:
fea_train=fea_train.reshape(27839, 32, 32, 1)

In [16]:
from keras.preprocessing.image import ImageDataGenerator

In [17]:
dataGen=ImageDataGenerator(rotation_range=10,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1)

In [18]:
dataGen.fit(fea_train)

In [19]:
batches=dataGen.flow(fea_train,tar_train,batch_size=20)

In [20]:
len(batches)

1392

In [21]:
images,labels=next(batches)

In [22]:
images.shape

(20, 32, 32, 1)

In [23]:
labels.shape

(20,)

In [24]:
from keras.utils import to_categorical

In [25]:
tar_train = to_categorical(tar_train)

In [26]:
tar_train.shape

(27839, 43)

In [27]:
fea_train.shape

(27839, 32, 32, 1)

In [28]:
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout
from keras.models import Sequential


In [29]:
model=Sequential()
model.add(Conv2D(60,(3,3),activation="relu",input_shape=(32,32,1)))
model.add(Conv2D(60,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(30,(3,3),activation="relu"))
model.add(Conv2D(30,(3,3),activation="relu"))
model.add(Conv2D(30,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dense(43,activation="softmax"))

In [30]:
from keras.optimizers import Adam

In [31]:
model.compile(Adam(lr=0.001),loss="categorical_crossentropy",metrics=["accuracy"])

In [32]:
model.fit_generator(dataGen.flow(fea_train,tar_train,batch_size=20),epochs=20)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
1392/1392 [==============================] - 69s 49ms/step - loss: 2.0416 - accuracy: 0.4296
Epoch 2/20
1392/1392 [==============================] - 69s 49ms/step - loss: 0.7140 - accuracy: 0.7857
Epoch 3/20
1392/1392 [==============================] - 70s 51ms/step - loss: 0.4684 - accuracy: 0.8550
Epoch 4/20
1392/1392 [==============================] - 77s 55ms/step - loss: 0.3491 - accuracy: 0.8936
Epoch 5/20
1392/1392 [==============================] - 73s 52ms/step - loss: 0.2933 - accuracy: 0.9084
Epoch 6/20
1392/1392 [==============================] - 71s 51ms/step - loss: 0.2529 - accuracy: 0.9221
Epoch 7/20
1392/1392 [==============================] - 73s 52ms/step - loss: 0.2314 - accuracy: 0.9293
Epoch 8/20
1392/1392 [==============================] - 70s 50ms/step - loss: 0.2094 - accuracy: 0.9356
Epoch 9/20
1392/1392 [==============================] - 67s 48ms/step - loss: 0.1877 - accur

In [33]:
from keras.models import model_from_json

In [34]:
model_json=model.to_json()
with open("verzeoAITraffic.json","w") as abc:
    abc.write(model_json)
    abc.close()
model.save_weights("verzeoAITrafficWeights.h5")
print("Save the Model")

Save the Model


In [35]:
json_file=open("verzeoAITraffic.json","r")
loaded_model_json=json_file.read()
json_file.close()
loaded_model=model_from_json(loaded_model_json)
loaded_model.load_weights("verzeoAITrafficWeights.h5")
print("Loaded Model Sucesssfully")

Loaded Model Sucesssfully


In [36]:
def getClassName(classNo):
    if   classNo == 0: return 'Speed Limit 20 km/h'
    elif classNo == 1: return 'Speed Limit 30 km/h'
    elif classNo == 2: return 'Speed Limit 50 km/h'
    elif classNo == 3: return 'Speed Limit 60 km/h'
    elif classNo == 4: return 'Speed Limit 70 km/h'
    elif classNo == 5: return 'Speed Limit 80 km/h'
    elif classNo == 6: return 'End of Speed Limit 80 km/h'
    elif classNo == 7: return 'Speed Limit 100 km/h'
    elif classNo == 8: return 'Speed Limit 120 km/h'
    elif classNo == 9: return 'No passing'
    elif classNo == 10: return 'No passing for vechiles over 3.5 metric tons'
    elif classNo == 11: return 'Right-of-way at the next intersection'
    elif classNo == 12: return 'Priority road'
    elif classNo == 13: return 'Yield'
    elif classNo == 14: return 'Stop'
    elif classNo == 15: return 'No vechiles'
    elif classNo == 16: return 'Vechiles over 3.5 metric tons prohibited'
    elif classNo == 17: return 'No entry'
    elif classNo == 18: return 'General caution'
    elif classNo == 19: return 'Dangerous curve to the left'
    elif classNo == 20: return 'Dangerous curve to the right'
    elif classNo == 21: return 'Double curve'
    elif classNo == 22: return 'Bumpy road'
    elif classNo == 23: return 'Slippery road'
    elif classNo == 24: return 'Road narrows on the right'
    elif classNo == 25: return 'Road work'
    elif classNo == 26: return 'Traffic signals'
    elif classNo == 27: return 'Pedestrians'
    elif classNo == 28: return 'Children crossing'
    elif classNo == 29: return 'Bicycles crossing'
    elif classNo == 30: return 'Beware of ice/snow'
    elif classNo == 31: return 'Wild animals crossing'
    elif classNo == 32: return 'End of all speed and passing limits'
    elif classNo == 33: return 'Turn right ahead'
    elif classNo == 34: return 'Turn left ahead'
    elif classNo == 35: return 'Ahead only'
    elif classNo == 36: return 'Go straight or right'
    elif classNo == 37: return 'Go straight or left'
    elif classNo == 38: return 'Keep right'
    elif classNo == 39: return 'Keep left'
    elif classNo == 40: return 'Roundabout mandatory'
    elif classNo == 41: return 'End of no passing'
    elif classNo == 42: return 'End of no passing by vechiles over 3.5 metric tons'

In [37]:
capt = cv2.VideoCapture(0)
capt.set(3,640)
capt.set(4,480)
capt.set(10,180)

False

In [1]:
while True:
    message,image=capt.read()
    imagearr=np.asarray(image)
    imagearr=cv2.resize(imagearr,(32,32))
    imagearr=preprocessing(imagearr)
    imagearr=imagearr.reshape(1,32,32,1)
    predictions=loaded_model.predict(imagearr)
    classIndex=loaded_model.predict_classes(imagearr)
    cv2.putText(image,"Class: ",(20,35),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    cv2.putText(image,"Probability: ",(20,75),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    probabilityValue=np.amax(predictions)
    if probabilityValue>0.75:
        cv2.putText(image,getClassName(classIndex),(120 , 35),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
        cv2.putText(image,str(int(probabilityValue * 100)) + " %",( 200,75),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    cv2.imshow("Model Prediction",image)
    returnedValue=cv2.waitKey(1)
    if returnedValue==ord("s") or returnedValue==ord("S"):
        cv2.destroyAllWindows()
        break

NameError: name 'capt' is not defined